# Load the Pretrained Model and the Dataset

In [1]:
import sys

sys.path.append("../..")
sys.path.append("../../../")

In [2]:
import paddle
import paddlenlp
from paddlenlp.transformers import ErnieForSequenceClassification, ErnieTokenizer

MODEL_NAME = 'ernie-1.0'

model = ErnieForSequenceClassification.from_pretrained(MODEL_NAME, num_classes=2)
tokenizer = ErnieTokenizer.from_pretrained(MODEL_NAME)

[2022-06-07 14:37:36,898] [    INFO] - Already cached /home/zhangshuai/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
W0607 14:37:36.901656 28872 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.4, Runtime API Version: 10.2
W0607 14:37:36.907972 28872 device_context.cc:465] device: 0, cuDNN Version: 8.2.
[2022-06-07 14:37:41,821] [    INFO] - Already cached /home/zhangshuai/.paddlenlp/models/ernie-1.0/vocab.txt


In [3]:
from paddlenlp.datasets import load_dataset

DATASET_NAME = 'chnsenticorp'
train_ds, dev_ds, test_ds = load_dataset(DATASET_NAME, splits=["train", "dev", "test"])

# Prepare the Model
## Train the model

In [ ]:
from assets.utils import training_model

training_model(model, tokenizer, train_ds, dev_ds, save_dir=f'../../assets/{DATASET_NAME}-{MODEL_NAME}')


## Or Load the trained model

In [4]:
# Load the trained model.
!wget --no-check-certificate -c https://trustai.bj.bcebos.com/chnsenticorp-ernie-1.0.tar
!tar -xvf ./chnsenticorp-ernie-1.0.tar -C ../../assets/
!rm ./chnsenticorp-ernie-1.0.tar

state_dict = paddle.load(f'../../assets/{DATASET_NAME}-{MODEL_NAME}/model_state.pdparams')
model.set_dict(state_dict)

--2022-06-07 14:37:41--  https://trustai.bj.bcebos.com/chnsenticorp-ernie-1.0.tar
Resolving trustai.bj.bcebos.com (trustai.bj.bcebos.com)... 10.70.0.165
Connecting to trustai.bj.bcebos.com (trustai.bj.bcebos.com)|10.70.0.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 399595520 (381M) [application/x-tar]
Saving to: ‘chnsenticorp-ernie-1.0.tar’

100%[======================================>] 399,595,520  120MB/s   in 3.2s   

2022-06-07 14:37:45 (120 MB/s) - ‘chnsenticorp-ernie-1.0.tar’ saved [399595520/399595520]

chnsenticorp-ernie-1.0/
chnsenticorp-ernie-1.0/tokenizer_config.json
chnsenticorp-ernie-1.0/vocab.txt
chnsenticorp-ernie-1.0/model_state.pdparams
chnsenticorp-ernie-1.0/model_config.json


# See the prediciton results

In [5]:
from assets.utils import predict

test_data = [{'text': '没有光驱,重装Windows需要外接光驱,对于电脑新手会很麻烦(没什么人会用Linux吧)'}]

label_map = {0: 'negative', 1: 'positive'}

batch_size = 32

results = predict(
    model, test_data, tokenizer, label_map, batch_size=batch_size)

for idx, text in enumerate(test_data):
    print('Data: {} \t Lable: {}'.format(text, results[idx]))

Data: {'text': '没有光驱,重装Windows需要外接光驱,对于电脑新手会很麻烦(没什么人会用Linux吧)'} 	 Lable: negative


# Prepare for Interpretations

In [6]:
from functools import partial

from paddlenlp.data import Stack, Tuple, Pad

from assets.utils import create_dataloader, convert_example


batch_size = 1 # attention, batch_size must be 1
max_seq_length = 128

trans_func = partial(convert_example,
                     tokenizer=tokenizer,
                     max_seq_length=max_seq_length,
                     is_test=True)
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # segment
): [data for data in fn(samples)]
train_data_loader = create_dataloader(train_ds,
                                      mode='train',
                                      batch_size=batch_size,
                                      batchify_fn=batchify_fn,
                                      trans_fn=trans_func,
                                      shuffle=False)

# Gradient similarity Interpreter

In [7]:
from trustai.interpretation import GradientSimilarityModel

# classifier_layer_name is the layer name of the last output layer
grad_sim = GradientSimilarityModel(model, train_data_loader, classifier_layer_name='classifier')

Extracting gradient for given dataloader, it will take some time...


In gradient similarity, Similarity function supports `cos` and `dot`, namely `Cosine simmilarity` and `Dot product`.

Using Cosine simmilarity:

In [8]:
from assets.utils import create_dataloader_from_scratch, predict, print_result

# process text to model input
test_dataloader = create_dataloader_from_scratch(test_data, tokenizer)

sim_fn = "cos"
sample_num = 3
res = grad_sim.interpret(test_dataloader, sample_num=sample_num, sim_fn=sim_fn)

print_result(test_data, train_ds, res, data_name='chnsenticorp')

Extracting gradient for given dataloader, it will take some time...
test data
text: 没有光驱,重装Windows需要外接光驱,对于电脑新手会很麻烦(没什么人会用Linux吧)	predict label: 0
examples with positive influence
text: Linux系统不太好用,平时习惯用Windows xp 系统,一下子用这个系统感觉很不习惯,建议开发或预装Windows xp系统.	gold label: 0	score: 0.9395108222961426
text: 1、机器较沉 2、VISTA用起来不习惯，且占系统较多 3、音频插口、右前侧的二个USB口在用鼠标时感觉手靠得太近了	gold label: 0	score: 0.9355786442756653
text: vista系统下也没有无线网卡驱动，用驱动精灵可解决。 机器稍有点重。 散热有待改进。	gold label: 0	score: 0.9349631071090698
examples with negative influence
text: 价格确实比较高，而且还没有早餐提供。 携程拿到的价格不好？还是自己保留起来不愿意让利给我们这些客户呢？ 到前台搞价格，430就可以了。	gold label: 1	score: -0.49774348735809326
text: 买机器送的移动硬盘2.5寸250G的，没开封，想卖出，感兴趣短息联系，北京13901019711	gold label: 1	score: -0.5244823694229126
text: 买机器送的移动硬盘2.5寸250G的，没开封，想卖出，感兴趣短息联系，北京13901019711	gold label: 0	score: -0.5244823694229126



Using Dot product:

In [9]:
sim_fn = "dot"

res = grad_sim.interpret(test_dataloader, sample_num=sample_num, sim_fn=sim_fn)

print_result(test_data, train_ds, res, data_name='chnsenticorp')

Extracting gradient for given dataloader, it will take some time...
test data
text: 没有光驱,重装Windows需要外接光驱,对于电脑新手会很麻烦(没什么人会用Linux吧)	predict label: 0
examples with positive influence
text: 看评论说有卖到4299还送瑞星杀毒，就是不知道是何时，如果消息属实，那确实会有点郁闷！	gold label: 0	score: 0.3056339621543884
text: 自己去松江配送点拿的货，一打看就看到显示屏下方的有一处黑漆掉色严重．是帮朋友买的，郁闷中．	gold label: 1	score: 0.2579595148563385
text: 建议酒店餐厅预备当地产的酒水,以备外地客人品尝.	gold label: 0	score: 0.2578386962413788
examples with negative influence
text: 价格确实比较高，而且还没有早餐提供。 携程拿到的价格不好？还是自己保留起来不愿意让利给我们这些客户呢？ 到前台搞价格，430就可以了。	gold label: 1	score: -0.24776840209960938
text: 买机器送的移动硬盘2.5寸250G的，没开封，想卖出，感兴趣短息联系，北京13901019711	gold label: 1	score: -0.2623222768306732
text: 买机器送的移动硬盘2.5寸250G的，没开封，想卖出，感兴趣短息联系，北京13901019711	gold label: 0	score: -0.2623222768306732

